# 04 · Validate Standards

In [6]:
from pathlib import Path
import pandas as pd, json

adam = Path.cwd().parents[0] / "cdisc" / "adam_like" / "adimmun.csv"
fhir = Path.cwd().parents[0] / "interoperability" / "fhir_examples"
out_md = Path.cwd().parents[0] / "validations" / "summary.md"
failures = []

req_cols = ["USUBJID","VISIT","METHOD","PARAMCD","ADASTAT","CONC_WK52_NGM"]
ad = pd.read_csv(adam)
missing = [c for c in req_cols if c not in ad.columns]
if missing: failures.append(f"ADaM missing columns: {missing}")
if (ad["CONC_WK52_NGM"] < 0).any(): failures.append("Negative concentrations present.")
if not set(ad["METHOD"]).issubset({"Standard","PandA"}): failures.append("Unclassified METHOD values found.")

jsons = list(fhir.glob("*.json"))
if not jsons: failures.append("No FHIR Observation JSON files found.")

with open(out_md, "w") as f:
    if failures:
        f.write("## Validation Summary: FAIL ❌\n\n")
        for e in failures: f.write(f"- {e}\n")
    else:
        f.write("## Validation Summary: PASS ✅\n\n- ADaM-like dataset structure valid\n- FHIR Observations present and minimally valid\n")
print("Wrote validation summary →", out_md)

Wrote validation summary → /Users/cmontefusco/Coding projects/dmpk-standards-interoperability/validations/summary.md
